In [1]:
if 'google.colab' in str(get_ipython()):
    !pip install pytorch-lifestream

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 KB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pandas as pd
import torch

In [3]:
if not os.path.exists('data/transactions.csv'):
    ! mkdir -p data
    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/0433a4ca/transactions.zip
    ! unzip -j -o transactions.zip '*.csv' -d data
    ! mv transactions.zip data/

if not os.path.exists('data/education.csv'):
    ! mkdir -p data
    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/e756bf99/train.csv
    #! unzip -j -o transactions.zip '*.csv' -d data
    ! mv train.csv data/


print(f'Loaded csv files')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  250M  100  250M    0     0  14.3M      0  0:00:17  0:00:17 --:--:-- 14.6M
Archive:  transactions.zip
  inflating: data/transactions.csv   
  inflating: data/._transactions.csv  
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  307k  100  307k    0     0   210k      0  0:00:01  0:00:01 --:--:--  210k
Loaded csv files


In [4]:
def trx_types(df):
    df['mcc_code'] = df['mcc_code'].astype(str)
    df['currency_rk'] = df['currency_rk'].astype(str)
    df['event_time'] = pd.to_datetime(df['transaction_dttm']).astype(int) / 1e9
    return df[['user_id', 'event_time', 'mcc_code', 'currency_rk', 'transaction_amt']]


def click_types(df):
    df['event_time'] = pd.to_datetime(df['timestamp']).astype(int) / 1e9
    # df = pd.merge(df, click_categories, on='cat_id')
    df['cat_id'] = df['cat_id'].astype(str)
    return df[['user_id', 'event_time', 'cat_id', 'new_uid']]


In [5]:

data_path = 'data/'

In [6]:
%%time
target = pd.read_csv(os.path.join(data_path, 'train.csv'))

CPU times: user 10.2 ms, sys: 4.1 ms, total: 14.3 ms
Wall time: 19.4 ms


# Transactions preprocessing

In [7]:
%%time
transactions = pd.read_csv(os.path.join(data_path, 'transactions.csv'))
transactions = transactions.dropna()
transactions = trx_types(transactions)

CPU times: user 47 s, sys: 7.13 s, total: 54.1 s
Wall time: 54 s


In [8]:
transactions.head(2)

,user_id,event_time,mcc_code,currency_rk,transaction_amt
0,000932580e404dafbecd5916d4640938,1.596442e+09,5411,48,-361.07230
1,000932580e404dafbecd5916d4640938,1.596591e+09,5499,48,-137.31398


In [9]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor_trx = PandasDataPreprocessor(
    col_id='user_id',
    col_event_time='event_time',
    event_time_transformation='none',
    cols_category=["mcc_code", "currency_rk"],
    cols_numerical=["transaction_amt"],
    return_records=True,
)

In [10]:

%%time
dataset_transactions = preprocessor_trx.fit_transform(transactions)
print(f'Transactions features prepared')

Transactions features prepared
CPU times: user 50.3 s, sys: 8.83 s, total: 59.1 s
Wall time: 1min 1s


In [11]:
del preprocessor_trx
del transactions

# Encoding

In [12]:
dataset_transactions = sorted(dataset_transactions, key=lambda x: x['user_id'])

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset_transactions, test_size=0.2, random_state=42)

len(train), len(test)

(18026, 4507)

In [14]:
train[0].keys()

dict_keys(['user_id', 'event_time', 'mcc_code', 'currency_rk', 'transaction_amt'])

### Model definition

In [19]:
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={},
    embeddings={
        'currency_rk': {'in': 200, 'out': 48},
        'mcc_code': {'in': 100, 'out': 24}
    },
)


seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9),
)

In [20]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule

train_dl = PtlsDataModule(
    train_data=ColesDataset(
        MemoryMapDataset(
            data=train,
            i_filters=[
                SeqLenFilter(min_seq_len=25),
            ],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=25,
            cnt_max=200,
        ),
    ),
    train_num_workers=16,
    train_batch_size=256,
)

In [21]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [22]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 265 K 
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
265 K     Trainable params
0         Non-trainable params
265 K     Total params
1.063     Total estimated model params size (MB)


logger.version = 1
{'loss': tensor(141.3889), 'seq_len': tensor(112.6321)}
CPU times: user 4min 3s, sys: 37.4 s, total: 4min 41s
Wall time: 6min 57s


In [23]:
# torch.save(seq_encoder.state_dict(), "coles-emb.pt")

In [24]:
# embedding inference

from ptls.data_load.datasets import inference_data_loader

train_dl = inference_data_loader(train, num_workers=0, batch_size=256)
train_embeds = torch.vstack(trainer.predict(model, train_dl, ))

test_dl = inference_data_loader(test, num_workers=0, batch_size=256)
test_embeds = torch.vstack(trainer.predict(model, test_dl))

train_embeds.shape, test_embeds.shape

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loops/epoch/prediction_epoch_loop.py:175: UserWarning: Lightning couldn't infer the indices fetched for your dataloader.
  warning_cache.warn("Lightning couldn't infer the indices fetched for your dataloader.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


(torch.Size([18026, 256]), torch.Size([4507, 256]))

In [25]:
# join target and embeddings

df_target = pd.read_csv(os.path.join(data_path, 'train.csv'))
df_target.rename(columns={"bank": "user_id"}, inplace=True)
df_target = df_target.set_index('user_id')
df_target.rename(columns={"higher_education": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['user_id'] = [x['user_id'] for x in train]
# train_df = train_df.merge(df_target, how='left', on='user_id')
train_df = train_df.merge(df_target, on='user_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['user_id'] = [x['user_id'] for x in test]
# test_df = test_df.merge(df_target, how='left', on='user_id')
test_df = test_df.merge(df_target, on='user_id')
print(train_df.shape, test_df.shape)

(6838, 258) (1671, 258)


In [26]:

from sklearn.ensemble import RandomForestClassifier

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.7636146020347098

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
y_pred = clf.predict(x_test)
print("accuracy score:", accuracy_score(y_test, y_pred))
print("precision score:", precision_score(y_test, y_pred))
print("recall score:", recall_score(y_test, y_pred))
print("f1 score:", f1_score(y_test, y_pred))
print("roc auc_score:", roc_auc_score(y_test, y_pred))

accuracy score: 0.7636146020347098
precision score: 0.7899794097460535
recall score: 0.9282258064516129
f1 score: 0.8535409714497589
roc auc_score: 0.6091245041538806
